# Semaphore

In [57]:
@everywhere function waitP(pid, r, procs)
    println("Process ", pid, " is waiting");
    if r > 0
        r -= 1;
        println("Process ", pid, " is running");
        return r, procs;
    else
        push!(procs, pid);
        println("Sleeping process ", pid);
        return r, procs;
    end
end;

@everywhere function postV(r, procs)
    
    if length(procs) > 0
        pid = pop(procs);
        println("Process ", pid, " is posting");
        return r, procs;
    else
        r+=1;
        println("Semaphore is incremented. Value of r is ", r)
        return r, procs;
    end
end;

@everywhere r = 3;
@everywhere _procs = [];

In [58]:
@everywhere function P(r = r, procs = _procs)
    i = rand(1:100);
    r, procs = waitP(i, r, procs);
end

@everywhere function V(r = r, procs = _procs)
    println("V is called");
    r, procs = postV(r, procs);
end

In [59]:
using Random

In [60]:
tasksP = [P for i in 1:5];
tasksV = [V for i in 1:5];

tasks = [tasksP; tasksV];
tasks = shuffle(tasks);


In [28]:
using Distributed

addprocs(2)

2-element Vector{Int64}:
 2
 3

In [61]:
@everywhere function runFunc(f, r = r, procs = _procs)
    f(r, procs)
end;

In [62]:
pmap(runFunc, tasks)

      From worker 3:	V is called
      From worker 2:	V is called
      From worker 2:	Semaphore is incremented. Value of r is 4
      From worker 3:	Semaphore is incremented. Value of r is 4
      From worker 2:	V is called
      From worker 3:	V is called
      From worker 3:	Semaphore is incremented. Value of r is 4
      From worker 2:	Semaphore is incremented. Value of r is 4
      From worker 3:	Process 12 is waiting
      From worker 3:	Process 12 is running
      From worker 2:	Process 89 is waiting
      From worker 2:	Process 89 is running
      From worker 3:	Process 37 is waiting
      From worker 2:	V is called
      From worker 2:	Semaphore is incremented. Value of r is 4
      From worker 3:	Process 37 is running
      From worker 2:	Process 70 is waiting
      From worker 2:	Process 70 is running


10-element Vector{Tuple{Int64, Vector{Any}}}:
 (4, [])
 (4, [])
 (4, [])
 (4, [])
 (2, [])
 (2, [])
 (2, [])
 (4, [])
 (2, [])
 (2, [])

      From worker 3:	Process 94 is waiting
      From worker 3:	Process 94 is running


# Dining Philosophers Problem

In [ ]:
mutable struct Philosopher
    name::String
    hungry::Bool
    righthanded::Bool
    rightforkheld::Channel
    leftforkheld::Channel
    function Philosopher(name, leftfork, rightfork)
        this = new()
        this.name = name
        this.hungry = rand([false, true]) # not specified so start as either
        this.righthanded   = (name == "Aristotle") ? false : true
        this.leftforkheld  = leftfork
        this.rightforkheld = rightfork
        this
    end
end

In [ ]:
mutable struct FiveForkTable
    fork51::Channel
    fork12::Channel
    fork23::Channel
    fork34::Channel
    fork45::Channel    
    function FiveForkTable()
        this = new()
        this.fork51 = Channel(1); put!(this.fork51, "fork") # start with one fork per channel
        this.fork12 = Channel(1); put!(this.fork12, "fork") 
        this.fork23 = Channel(1); put!(this.fork23, "fork") 
        this.fork34 = Channel(1); put!(this.fork34, "fork") 
        this.fork45 = Channel(1); put!(this.fork45, "fork") 
        this
    end
end

In [ ]:
table = FiveForkTable();
tasks = [Philosopher("Aristotle", table.fork12, table.fork51),
         Philosopher("Kant", table.fork23, table.fork12),
         Philosopher("Spinoza", table.fork34, table.fork23),
         Philosopher("Marx", table.fork45, table.fork34),
         Philosopher("Russell", table.fork51, table.fork45)]

In [ ]:
function dine(t,p)
    if p.righthanded
       take!(p.rightforkheld); println("$(p.name) takes right fork")
       take!(p.leftforkheld); println("$(p.name) takes left fork")
    else
       take!(p.leftforkheld); println("$(p.name) takes left fork")
       take!(p.rightforkheld); println("$(p.name) takes right fork")
    end    
end

In [ ]:
function leavetothink(t, p)
    put!(p.rightforkheld, "fork"); println("$(p.name) puts down right fork")
    put!(p.leftforkheld, "fork");  println("$(p.name) puts down left fork")
end

In [ ]:
contemplate(t) = sleep(t)

In [ ]:
function dophil(p, t, fullaftersecs=2.0, hungryaftersecs=10.0)
    while true
        if p.hungry
            println("$(p.name) is hungry")
            dine(table, p)
            sleep(fullaftersecs)
            p.hungry = false
            leavetothink(t, p)
        else
            println("$(p.name) is out of the dining room for now.")
            contemplate(hungryaftersecs)
            p.hungry = true
        end
    end
end

In [ ]:
function runall(tasklist)
    for p in tasklist
        @async dophil(p, table)
    end
    while true begin sleep(5) end end
end

In [91]:
runall(tasks)

Aristotle is hungry
Aristotle takes left fork
Aristotle takes right fork
Marx is hungry
Marx takes right fork
Marx takes left fork
Aristotle puts down right fork
Aristotle puts down left fork
Aristotle is out of the dining room for now.
Marx puts down right fork
Marx puts down left fork
Marx is out of the dining room for now.
Kant is hungry
Kant takes right fork
Kant takes left fork
Russell is hungry
Russell takes right fork
Russell takes left fork
Spinoza is hungry
Kant puts down right fork
Kant puts down left fork
Kant is out of the dining room for now.
Spinoza takes right fork
Spinoza takes left fork
Russell puts down right fork
Russell puts down left fork
Russell is out of the dining room for now.
Spinoza puts down right fork
Spinoza puts down left fork
Spinoza is out of the dining room for now.
Aristotle is hungry
Aristotle takes left fork
Aristotle takes right fork
Marx is hungry
Marx takes right fork
Marx takes left fork
Aristotle puts down right fork
Aristotle puts down left fo